<a href="https://colab.research.google.com/github/noorwewe/Intelligent-System/blob/master/RBM_AE_DBN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np

# Define RBM parameters
num_visible = 6 # Number of visible units
num_hidden = 2 # Number of hidden units
learning_rate = 0.01
epochs = 500 # More epochs might be needed for backprop
batch_size = 10

# Sample data (binary)
data = np.array([
    [1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 0]
], dtype=np.float32)

# Initialize weights and biases
# Using different names to avoid conflict with previous RBM example
W_bp = tf.Variable(tf.random.normal([num_visible, num_hidden], mean=0.0, stddev=0.01))
vb_bp = tf.Variable(tf.zeros([num_visible])) # Visible bias
hb_bp = tf.Variable(tf.zeros([num_hidden]))  # Hidden bias

# RBM functions (using the same logic but different variables)
def forward_pass_bp(visible_layer):
    hidden_activations = tf.matmul(visible_layer, W_bp) + hb_bp
    hidden_probabilities = tf.sigmoid(hidden_activations)
    return hidden_probabilities

def backward_pass_bp(hidden_layer):
    visible_activations = tf.matmul(hidden_layer, tf.transpose(W_bp)) + vb_bp
    visible_probabilities = tf.sigmoid(visible_activations)
    return visible_probabilities

# Training function (Backpropagation for Reconstruction Error)
@tf.function
def train_step_bp(batch_data):
    with tf.GradientTape() as tape:
        # Forward pass to hidden layer
        hidden_probs = forward_pass_bp(batch_data)
        # Backward pass to reconstruct visible layer
        reconstructed_probs = backward_pass_bp(hidden_probs)

        # Compute Reconstruction Loss (Mean Squared Error)
        loss = tf.reduce_mean(tf.square(batch_data - reconstructed_probs))

    # Compute gradients and apply them
    gradients = tape.gradient(loss, [W_bp, vb_bp, hb_bp])
    return gradients, loss, reconstructed_probs

# Training loop
optimizer_bp = tf.optimizers.Adam(learning_rate)

print("Starting RBM training with Backpropagation...")
for epoch in range(epochs):
    total_loss = 0
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i+batch_size]
        gradients, loss, _ = train_step_bp(batch_data)
        optimizer_bp.apply_gradients(zip(gradients, [W_bp, vb_bp, hb_bp]))
        total_loss += loss.numpy()

    if (epoch + 1) % 50 == 0: # Print less frequently for more epochs
        print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')

print("Training finished.")

# Example reconstruction after training
sample_data_bp = np.array([[1, 1, 0, 0, 0, 0]], dtype=np.float32)
hidden_probs_bp = forward_pass_bp(sample_data_bp)
# For reconstruction after training, we typically use the probabilities directly
reconstruction_bp = backward_pass_bp(hidden_probs_bp)


print("\nOriginal data:")
print(sample_data_bp)
print("Reconstruction (using Backprop trained RBM):")
print(reconstruction_bp.numpy())

Starting RBM training with Backpropagation...
Epoch 50, Loss: 0.1679
Epoch 100, Loss: 0.0994
Epoch 150, Loss: 0.0559
Epoch 200, Loss: 0.0432
Epoch 250, Loss: 0.0367
Epoch 300, Loss: 0.0315
Epoch 350, Loss: 0.0271
Epoch 400, Loss: 0.0234
Epoch 450, Loss: 0.0204
Epoch 500, Loss: 0.0179
Training finished.

Original data:
[[1. 1. 0. 0. 0. 0.]]
Reconstruction (using Backprop trained RBM):
[[0.94361836 0.7975779  0.9488809  0.05284929 0.01449    0.05772183]]


In [6]:
# Use the trained RBM architecture with backpropagation for a simple autoencoder
# The encoder part is the forward pass of the RBM
# The decoder part is the backward pass of the RBM

# We will use the weights and biases learned from the RBM training (W_bp, vb_bp, hb_bp)
# Define the autoencoder model (using functional API for clarity)

# Create custom Keras layers for the RBM operations to work with Functional API
class RBMLayer_BP(tf.keras.layers.Layer):
    def __init__(self, weights, bias, activation=tf.sigmoid, **kwargs):
        super(RBMLayer_BP, self).__init__(**kwargs)
        self._weights = weights  # Store as internal variable
        self._bias = bias      # Store as internal variable
        self.activation = activation

    def build(self, input_shape):
        # Add weights and bias using add_weight, but set trainable=False
        self.kernel = self.add_weight(
            shape=self._weights.shape,
            initializer=tf.constant_initializer(self._weights.numpy()), # Use numpy value
            trainable=False,
            name='kernel'
        )
        self.bias_weight = self.add_weight(
            shape=self._bias.shape,
            initializer=tf.constant_initializer(self._bias.numpy()), # Use numpy value
            trainable=False,
            name='bias'
        )
        super(RBMLayer_BP, self).build(input_shape)

    def call(self, inputs):
        output_activations = tf.matmul(inputs, self.kernel) + self.bias_weight
        return self.activation(output_activations)

# Define the autoencoder model (using functional API)
visible_input_bp = tf.keras.Input(shape=(num_visible,), dtype=tf.float32) # Use num_visible from previous cell

# Encoder part (using RBM forward pass logic wrapped in a Layer)
# Reuse the trained weights and biases from the backprop RBM
hidden_layer_output_bp = RBMLayer_BP(weights=W_bp, bias=hb_bp, activation=tf.sigmoid, name='encoder_bp')(visible_input_bp)

# Decoder part (using RBM backward pass logic wrapped in a Layer)
# Reuse the trained weights (transposed) and visible bias from the backprop RBM
reconstructed_output_bp = RBMLayer_BP(weights=tf.transpose(W_bp), bias=vb_bp, activation=tf.sigmoid, name='decoder_bp')(hidden_layer_output_bp)

# Create the autoencoder model
autoencoder_bp = tf.keras.Model(inputs=visible_input_bp, outputs=reconstructed_output_bp)

# Create a model to get the hidden layer output (encoder model)
encoder_model_bp = tf.keras.Model(inputs=visible_input_bp, outputs=hidden_layer_output_bp)

print("Autoencoder Model Summary (using Backprop RBM weights):")
autoencoder_bp.summary()

print("\nEncoder Model Summary (using Backprop RBM weights):")
encoder_model_bp.summary()

# Example data to pass through the autoencoder and get hidden representation
# Use the same sample data as before for consistency
sample_data_autoencoder_bp = np.array([
    [1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [1, 0, 0, 0, 0, 0] # New sample
], dtype=np.float32)

# Get the hidden layer representation using the encoder model
hidden_representation_bp = encoder_model_bp.predict(sample_data_autoencoder_bp)

print("\nOriginal Data Samples:")
print(sample_data_autoencoder_bp)

print("\nHidden Layer Representation (from Autoencoder Encoder using Backprop RBM weights):")
print(hidden_representation_bp)

# Get reconstruction from the autoencoder
reconstructed_data_bp = autoencoder_bp.predict(sample_data_autoencoder_bp)
print("\nReconstructed Data (from Autoencoder using Backprop RBM weights):")
print(reconstructed_data_bp)

Autoencoder Model Summary (using Backprop RBM weights):


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bp (RBMLayer_BP)        │ (None, 2)              │            14 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_bp (RBMLayer_BP)        │ (None, 6)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32 (128.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bp (RBMLayer_BP)        │ (None, 2)              │            14 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14 (56.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14 (56.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step

Original Data Samples:
[[1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]

Hidden Layer Representation (from Autoencoder Encoder using Backprop RBM weights):
[[0.03438778 0.9996776 ]
 [0.9993805  0.02129759]
 [0.23454928 0.99218756]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step

Reconstructed Data (from Autoencoder using Backprop RBM weights):
[[0.94029075 0.78157395 0.950906   0.05678039 0.01506606 0.05563642]
 [0.03927356 0.0195979  0.950175   0.92803425 0.7803576  0.05698124]
 [0.9038879  0.62851524 0.962958   0.09976405 0.02145548 0.04305497]]


In [7]:
import tensorflow as tf
import numpy as np

# Use parameters from previous cells
# num_visible (from RBM input), num_hidden (for first RBM)
# data (from RBM input)
# learning_rate, epochs, batch_size (can reuse or redefine)

# Parameters for the first RBM (Visible to Hidden 1)
num_visible_layer1 = num_visible # Input layer size
num_hidden_layer1 = num_hidden   # Size of the first hidden layer (same as RBM example)
learning_rate_layer1 = 0.01
epochs_layer1 = 100 # Can adjust epochs for each layer
batch_size_layer1 = 10

# Parameters for the second RBM (Hidden 1 to Hidden 2)
num_visible_layer2 = num_hidden_layer1 # Input to second RBM is output of first hidden layer
num_hidden_layer2 = 5 # Size of the second hidden layer (choose a new size)
learning_rate_layer2 = 0.01
epochs_layer2 = 100
batch_size_layer2 = 10

# --- Training the First RBM Layer ---

print("--- Training First RBM Layer ---")

# Initialize weights and biases for the first RBM
W1 = tf.Variable(tf.random.normal([num_visible_layer1, num_hidden_layer1], mean=0.0, stddev=0.01))
vb1 = tf.Variable(tf.zeros([num_visible_layer1])) # Visible bias for layer 1
hb1 = tf.Variable(tf.zeros([num_hidden_layer1]))  # Hidden bias for layer 1

# RBM functions for Layer 1 (using CD)
def forward_pass_layer1(visible_layer):
    hidden_activations = tf.matmul(visible_layer, W1) + hb1
    hidden_probabilities = tf.sigmoid(hidden_activations)
    return hidden_probabilities

def backward_pass_layer1(hidden_layer):
    visible_activations = tf.matmul(hidden_layer, tf.transpose(W1)) + vb1
    visible_probabilities = tf.sigmoid(visible_activations)
    return visible_probabilities

@tf.function
def train_step_layer1(batch_data):
    with tf.GradientTape() as tape:
        positive_hidden_probs = forward_pass_layer1(batch_data)
        hidden_states = tf.floor(positive_hidden_probs + tf.random.uniform(tf.shape(positive_hidden_probs)))
        negative_visible_probs = backward_pass_layer1(hidden_states)
        negative_hidden_probs = forward_pass_layer1(negative_visible_probs)

        loss = tf.reduce_mean(tf.square(batch_data - negative_visible_probs))

    gradients = tape.gradient(loss, [W1, vb1, hb1])
    return gradients, loss

optimizer_layer1 = tf.optimizers.Adam(learning_rate_layer1)

for epoch in range(epochs_layer1):
    total_loss = 0
    for i in range(0, len(data), batch_size_layer1):
        batch_data = data[i:i+batch_size_layer1]
        gradients, loss = train_step_layer1(batch_data)
        optimizer_layer1.apply_gradients(zip(gradients, [W1, vb1, hb1]))
        total_loss += loss.numpy()

    if (epoch + 1) % 20 == 0:
        print(f'Layer 1 Epoch {epoch+1}, Loss: {total_loss:.4f}')

print("First RBM Layer Training finished.")

# --- Preparing Data for the Second RBM Layer ---
# The input data for the second RBM is the hidden representation from the first RBM
# For DBNs, we typically use the sampled binary states or the probabilities
# Let's use the probabilities for simplicity here

# Process the entire dataset through the trained first RBM to get hidden representations
hidden_representations_layer1 = forward_pass_layer1(data)

# --- Training the Second RBM Layer ---

print("\n--- Training Second RBM Layer ---")

# Initialize weights and biases for the second RBM
W2 = tf.Variable(tf.random.normal([num_visible_layer2, num_hidden_layer2], mean=0.0, stddev=0.01))
vb2 = tf.Variable(tf.zeros([num_visible_layer2])) # Visible bias for layer 2 (input is H1)
hb2 = tf.Variable(tf.zeros([num_hidden_layer2]))  # Hidden bias for layer 2

# RBM functions for Layer 2 (using CD)
def forward_pass_layer2(visible_layer):
    hidden_activations = tf.matmul(visible_layer, W2) + hb2
    hidden_probabilities = tf.sigmoid(hidden_activations)
    return hidden_probabilities

def backward_pass_layer2(hidden_layer):
    visible_activations = tf.matmul(hidden_layer, tf.transpose(W2)) + vb2
    visible_probabilities = tf.sigmoid(visible_activations)
    return visible_probabilities

@tf.function
def train_step_layer2(batch_data_h1): # Batch data from hidden layer 1
    with tf.GradientTape() as tape:
        positive_hidden_probs = forward_pass_layer2(batch_data_h1)
        hidden_states = tf.floor(positive_hidden_probs + tf.random.uniform(tf.shape(positive_hidden_probs)))
        negative_visible_probs = backward_pass_layer2(hidden_states)
        negative_hidden_probs = forward_pass_layer2(negative_visible_probs)

        loss = tf.reduce_mean(tf.square(batch_data_h1 - negative_visible_probs))

    gradients = tape.gradient(loss, [W2, vb2, hb2])
    return gradients, loss

optimizer_layer2 = tf.optimizers.Adam(learning_rate_layer2)

for epoch in range(epochs_layer2):
    total_loss = 0
    # Train the second RBM on the hidden representations from the first RBM
    for i in range(0, len(hidden_representations_layer1), batch_size_layer2):
        batch_data_h1 = hidden_representations_layer1[i:i+batch_size_layer2]
        gradients, loss = train_step_layer2(batch_data_h1)
        optimizer_layer2.apply_gradients(zip(gradients, [W2, vb2, hb2]))
        total_loss += loss.numpy()

    if (epoch + 1) % 20 == 0:
        print(f'Layer 2 Epoch {epoch+1}, Loss: {total_loss:.4f}')

print("Second RBM Layer Training finished.")

# --- Using the DBN for Feature Extraction ---
# The DBN encoder consists of the forward passes of the trained RBMs

def dbn_forward(visible_layer):
    # Pass through the first RBM's forward pass
    hidden_layer1_output = forward_pass_layer1(visible_layer)
    # Pass the output of the first hidden layer through the second RBM's forward pass
    hidden_layer2_output = forward_pass_layer2(hidden_layer1_output)
    return hidden_layer2_output # This is the highest-level feature representation

# Example using the DBN to get features
sample_data_dbn = np.array([
    [1, 1, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [1, 0, 0, 0, 0, 0]
], dtype=np.float32)

dbn_features = dbn_forward(sample_data_dbn)

print("\nOriginal Data Samples:")
print(sample_data_dbn)

print("\nDBN Highest-Level Features (from Layer 2 Hidden):")
print(dbn_features.numpy())

--- Training First RBM Layer ---


/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:855: UserWarning: Gradients do not exist for variables ['Variable:0'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Layer 1 Epoch 20, Loss: 0.2137
Layer 1 Epoch 40, Loss: 0.1904
Layer 1 Epoch 60, Loss: 0.1827
Layer 1 Epoch 80, Loss: 0.1763
Layer 1 Epoch 100, Loss: 0.1690
First RBM Layer Training finished.

--- Training Second RBM Layer ---
Layer 2 Epoch 20, Loss: 0.0044
Layer 2 Epoch 40, Loss: 0.0023
Layer 2 Epoch 60, Loss: 0.0034
Layer 2 Epoch 80, Loss: 0.0044
Layer 2 Epoch 100, Loss: 0.0033
Second RBM Layer Training finished.

Original Data Samples:
[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]

DBN Highest-Level Features (from Layer 2 Hidden):
[[0.5132458  0.5256634  0.51397353 0.51642644 0.51589555]
 [0.52258587 0.54440266 0.524032   0.52808845 0.5271433 ]
 [0.5157002  0.5305383  0.51660025 0.5194848  0.5188482 ]]


In [9]:
import tensorflow as tf
import numpy as np

# Reuse the DBN architecture and trained weights from the previous cell (Z8TVH_wyN-2m)
# W1, vb1, hb1 (for Layer 1)
# W2, vb2, hb2 (for Layer 2)
# dbn_forward function

# Define a simple binary classification dataset
# We'll create some synthetic labels for the existing data
classification_data = data # Use the same data as for RBM/DBN training
# Create synthetic binary labels (e.g., based on the first feature)
# If the first feature is 1, label is 1, otherwise 0
classification_labels = np.array([[1] if row[0] == 1 else [0] for row in classification_data], dtype=np.float32)

print("Classification Data:")
print(classification_data)
print("\nClassification Labels:")
print(classification_labels)

# --- Building the Classification Model ---
# We will use the DBN's forward pass as the feature extractor and add a classification layer

# Input layer
classification_input = tf.keras.Input(shape=(num_visible_layer1,), dtype=tf.float32)

# DBN Feature Extractor (using the trained weights)
# Since dbn_forward is a Python function operating on tensors, we can wrap it
# or define the layers explicitly using the trained weights.
# Let's define it explicitly using custom layers similar to the autoencoder example,
# but without making them Keras Layers if we just use them sequentially.
# Alternatively, wrap the dbn_forward in a Keras Layer for Functional API

class DBNFeatureExtractor(tf.keras.layers.Layer):
    def __init__(self, W1, vb1, hb1, W2, vb2, hb2, **kwargs):
        super(DBNFeatureExtractor, self).__init__(**kwargs)
        # Store weights and biases. In a real scenario, these would be passed from the trained RBMs
        self._W1 = W1
        self._vb1 = vb1
        self._hb1 = hb1
        self._W2 = W2
        self._vb2 = vb2
        self._hb2 = hb2

    def call(self, inputs):
        # First RBM layer forward pass
        hidden_layer1_activations = tf.matmul(inputs, self._W1) + self._hb1
        hidden_layer1_output = tf.sigmoid(hidden_layer1_activations)

        # Second RBM layer forward pass
        hidden_layer2_activations = tf.matmul(hidden_layer1_output, self._W2) + self._hb2
        hidden_layer2_output = tf.sigmoid(hidden_layer2_activations) # This is the DBN feature

        return hidden_layer2_output

# Instantiate the DBN feature extractor layer using the trained weights
dbn_features_output = DBNFeatureExtractor(W1=W1, vb1=vb1, hb1=hb1, W2=W2, vb2=vb2, hb2=hb2, name='dbn_features')(classification_input)

# Classification layer (Dense layer with sigmoid activation for binary classification)
classification_output = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(dbn_features_output)

# Create the classification model
classification_model = tf.keras.Model(inputs=classification_input, outputs=classification_output)

# Compile the model
classification_model.compile(optimizer='adam',
                             loss='binary_crossentropy',
                             metrics=['accuracy'])

print("\nClassification Model Summary:")
classification_model.summary()

# --- Training the Classification Model ---
print("\n--- Training Classification Model ---")

epochs_classifier = 1000 # Epochs for training the classifier layer
batch_size_classifier = 10

history = classification_model.fit(classification_data, classification_labels,
                                   epochs=epochs_classifier,
                                   batch_size=batch_size_classifier,
                                   verbose=1) # Set verbose to 1 to see training progress

print("Classification Model Training finished.")

# --- Evaluation ---
loss, accuracy = classification_model.evaluate(classification_data, classification_labels, verbose=0)
print(f'\nClassification Accuracy on training data: {accuracy:.4f}')

# --- Make Predictions ---
sample_data_classify = np.array([
    [1, 1, 0, 0, 0, 0], # Should be class 1
    [0, 0, 1, 1, 1, 0], # Should be class 0 (based on our synthetic labels)
    [1, 0, 0, 0, 0, 0]  # Should be class 1
], dtype=np.float32)

predictions = classification_model.predict(sample_data_classify)

print("\nSample Data for Classification:")
print(sample_data_classify)
print("\nPredictions (Probability of Class 1):")
print(predictions)

# Convert probabilities to binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)
print("\nBinary Predictions:")
print(binary_predictions)

Classification Data:
[[1. 1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0.]]

Classification Labels:
[[1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]]

Classification Model Summary:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dbn_features                    │ (None, 5)              │             0 │
│ (DBNFeatureExtractor)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)


--- Training Classification Model ---
Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step - accuracy: 0.5000 - loss: 0.7411
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 0.7406
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 0.7400
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 0.7395
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 0.7390
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5000 - loss: 0.7384
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.5000 - loss: 0.7379
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5000 - loss: 0.7374
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5000 - loss: 0.7368
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.7363
Epoch 11/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 0.7358
Epoch 12/1000
1/1 ━━━━

In [10]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# --- Generate a new synthetic binary dataset ---
print("--- Generating a new synthetic binary dataset ---")

# Define parameters for the new dataset
num_samples_new = 100
num_features_new = 10 # Increase number of features
# Create a synthetic dataset with some underlying pattern
# Let's make the label depend on the sum of the first 5 features
new_data = np.random.randint(0, 2, size=(num_samples_new, num_features_new)).astype(np.float32)
new_labels = np.array([[1] if np.sum(row[:5]) >= 3 else [0] for row in new_data], dtype=np.float32) # Synthetic labels

print(f"New Dataset Shape: {new_data.shape}")
print(f"New Labels Shape: {new_labels.shape}")

# Split the new dataset into training and testing sets
train_data_new, test_data_new, train_labels_new, test_labels_new = train_test_split(
    new_data, new_labels, test_size=0.2, random_state=42)

print(f"\nTraining Data Shape: {train_data_new.shape}")
print(f"Testing Data Shape: {test_data_new.shape}")


# --- Adapt DBN Classification Code for the new dataset ---
# Reuse the DBN architecture concept, but adapt to new input size
# We'll redefine the DBN/Classifier model to match the new data shape

# Parameters for the first RBM Layer (Visible to Hidden 1)
num_visible_layer1_new = num_features_new # Input layer size is now num_features_new
num_hidden_layer1_new = 5 # Choose a suitable size for the first hidden layer
learning_rate_layer1_new = 0.01
epochs_layer1_new = 200 # Adjust epochs
batch_size_layer1_new = 10

# Parameters for the second RBM Layer (Hidden 1 to Hidden 2)
num_visible_layer2_new = num_hidden_layer1_new # Input to second RBM is output of first hidden layer
num_hidden_layer2_new = 3 # Choose a suitable size for the second hidden layer
learning_rate_layer2_new = 0.01
epochs_layer2_new = 200
batch_size_layer2_new = 10

# --- Training the First RBM Layer on New Data ---

print("\n--- Training First RBM Layer on New Data ---")

# Initialize weights and biases for the first RBM (new shapes)
W1_new = tf.Variable(tf.random.normal([num_visible_layer1_new, num_hidden_layer1_new], mean=0.0, stddev=0.01))
vb1_new = tf.Variable(tf.zeros([num_visible_layer1_new])) # Visible bias for layer 1
hb1_new = tf.Variable(tf.zeros([num_hidden_layer1_new]))  # Hidden bias for layer 1

# RBM functions for Layer 1 (using CD) - adapted for new variables
def forward_pass_layer1_new(visible_layer):
    hidden_activations = tf.matmul(visible_layer, W1_new) + hb1_new
    hidden_probabilities = tf.sigmoid(hidden_activations)
    return hidden_probabilities

def backward_pass_layer1_new(hidden_layer):
    visible_activations = tf.matmul(hidden_layer, tf.transpose(W1_new)) + vb1_new
    visible_probabilities = tf.sigmoid(visible_activations)
    return visible_probabilities

@tf.function
def train_step_layer1_new(batch_data):
    with tf.GradientTape() as tape:
        positive_hidden_probs = forward_pass_layer1_new(batch_data)
        hidden_states = tf.floor(positive_hidden_probs + tf.random.uniform(tf.shape(positive_hidden_probs)))
        negative_visible_probs = backward_pass_layer1_new(hidden_states)
        negative_hidden_probs = forward_pass_layer1_new(negative_visible_probs)
        loss = tf.reduce_mean(tf.square(batch_data - negative_visible_probs))
    gradients = tape.gradient(loss, [W1_new, vb1_new, hb1_new])
    return gradients, loss

optimizer_layer1_new = tf.optimizers.Adam(learning_rate_layer1_new)

for epoch in range(epochs_layer1_new):
    total_loss = 0
    for i in range(0, len(train_data_new), batch_size_layer1_new):
        batch_data = train_data_new[i:i+batch_size_layer1_new]
        gradients, loss = train_step_layer1_new(batch_data)
        optimizer_layer1_new.apply_gradients(zip(gradients, [W1_new, vb1_new, hb1_new]))
        total_loss += loss.numpy()
    if (epoch + 1) % 50 == 0:
        print(f'Layer 1 Epoch {epoch+1}, Loss: {total_loss:.4f}')
print("First RBM Layer Training finished.")

# --- Preparing Data for the Second RBM Layer on New Data ---
hidden_representations_layer1_new = forward_pass_layer1_new(train_data_new)

# --- Training the Second RBM Layer on New Data ---

print("\n--- Training Second RBM Layer on New Data ---")

# Initialize weights and biases for the second RBM (new shapes)
W2_new = tf.Variable(tf.random.normal([num_visible_layer2_new, num_hidden_layer2_new], mean=0.0, stddev=0.01))
vb2_new = tf.Variable(tf.zeros([num_visible_layer2_new])) # Visible bias for layer 2 (input is H1)
hb2_new = tf.Variable(tf.zeros([num_hidden_layer2_new]))  # Hidden bias for layer 2

# RBM functions for Layer 2 (using CD) - adapted for new variables
def forward_pass_layer2_new(visible_layer):
    hidden_activations = tf.matmul(visible_layer, W2_new) + hb2_new
    hidden_probabilities = tf.sigmoid(hidden_activations)
    return hidden_probabilities

def backward_pass_layer2_new(hidden_layer):
    visible_activations = tf.matmul(hidden_layer, tf.transpose(W2_new)) + vb2_new
    visible_probabilities = tf.sigmoid(visible_activations)
    return visible_probabilities

@tf.function
def train_step_layer2_new(batch_data_h1): # Batch data from hidden layer 1
    with tf.GradientTape() as tape:
        positive_hidden_probs = forward_pass_layer2_new(batch_data_h1)
        hidden_states = tf.floor(positive_hidden_probs + tf.random.uniform(tf.shape(positive_hidden_probs)))
        negative_visible_probs = backward_pass_layer2_new(hidden_states)
        negative_hidden_probs = forward_pass_layer2_new(negative_visible_probs)
        loss = tf.reduce_mean(tf.square(batch_data_h1 - negative_visible_probs))
    gradients = tape.gradient(loss, [W2_new, vb2_new, hb2_new])
    return gradients, loss

optimizer_layer2_new = tf.optimizers.Adam(learning_rate_layer2_new)

for epoch in range(epochs_layer2_new):
    total_loss = 0
    for i in range(0, len(hidden_representations_layer1_new), batch_size_layer2_new):
        batch_data_h1 = hidden_representations_layer1_new[i:i+batch_size_layer2_new]
        gradients, loss = train_step_layer2_new(batch_data_h1)
        optimizer_layer2_new.apply_gradients(zip(gradients, [W2_new, vb2_new, hb2_new]))
        total_loss += loss.numpy()
    if (epoch + 1) % 50 == 0:
        print(f'Layer 2 Epoch {epoch+1}, Loss: {total_loss:.4f}')
print("Second RBM Layer Training finished.")

# --- Building the DBN Classification Model for New Data ---

print("\n--- Building and Training DBN Classification Model for New Data ---")

# DBN Feature Extractor for new data (using the trained weights)
class DBNFeatureExtractor_New(tf.keras.layers.Layer):
    def __init__(self, W1, vb1, hb1, W2, vb2, hb2, **kwargs):
        super(DBNFeatureExtractor_New, self).__init__(**kwargs)
        self._W1 = W1
        self._vb1 = vb1
        self._hb1 = hb1
        self._W2 = W2
        self._vb2 = vb2
        self._hb2 = hb2

    def call(self, inputs):
        hidden_layer1_activations = tf.matmul(inputs, self._W1) + self._hb1
        hidden_layer1_output = tf.sigmoid(hidden_layer1_activations)
        hidden_layer2_activations = tf.matmul(hidden_layer1_output, self._W2) + self._hb2
        hidden_layer2_output = tf.sigmoid(hidden_layer2_activations)
        return hidden_layer2_output

# Input layer for classification model (matches new data feature size)
classification_input_new = tf.keras.Input(shape=(num_features_new,), dtype=tf.float32)

# Instantiate the new DBN feature extractor layer using the trained weights
dbn_features_output_new = DBNFeatureExtractor_New(W1=W1_new, vb1=vb1_new, hb1=hb1_new, W2=W2_new, vb2=vb2_new, hb2=hb2_new, name='dbn_features_new')(classification_input_new)

# Classification layer
classification_output_new = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier_new')(dbn_features_output_new)

# Create the new classification model
classification_model_new = tf.keras.Model(inputs=classification_input_new, outputs=classification_output_new)

# Compile the new model
classification_model_new.compile(optimizer='adam',
                                 loss='binary_crossentropy',
                                 metrics=['accuracy'])

print("\nNew Classification Model Summary:")
classification_model_new.summary()

# --- Training the Classification Model on New Data ---
epochs_classifier_new = 100 # Adjust epochs for classifier
batch_size_classifier_new = 10

history_new = classification_model_new.fit(train_data_new, train_labels_new,
                                           epochs=epochs_classifier_new,
                                           batch_size=batch_size_classifier_new,
                                           verbose=1,
                                           validation_data=(test_data_new, test_labels_new)) # Use test data for validation

print("\nNew Classification Model Training finished.")

# --- Evaluation on New Data ---
loss_new, accuracy_new = classification_model_new.evaluate(test_data_new, test_labels_new, verbose=0)
print(f'\nClassification Accuracy on new test data: {accuracy_new:.4f}')

# --- Make Predictions on New Data ---
sample_data_classify_new = np.array([
    np.random.randint(0, 2, size=(num_features_new,)).astype(np.float32) for _ in range(5) # Generate 5 random samples
], dtype=np.float32)

predictions_new = classification_model_new.predict(sample_data_classify_new)

print("\nSample Data for Classification (New Dataset):")
print(sample_data_classify_new)
print("\nPredictions (Probability of Class 1) on New Dataset:")
print(predictions_new)

# Convert probabilities to binary predictions (0 or 1)
binary_predictions_new = (predictions_new > 0.5).astype(int)
print("\nBinary Predictions on New Dataset:")
print(binary_predictions_new)

--- Generating a new synthetic binary dataset ---
New Dataset Shape: (100, 10)
New Labels Shape: (100, 1)

Training Data Shape: (80, 10)
Testing Data Shape: (20, 10)

--- Training First RBM Layer on New Data ---


/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:855: UserWarning: Gradients do not exist for variables ['Variable:0'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Layer 1 Epoch 50, Loss: 1.9543
Layer 1 Epoch 100, Loss: 1.8148
Layer 1 Epoch 150, Loss: 1.7400
Layer 1 Epoch 200, Loss: 1.7197
First RBM Layer Training finished.

--- Training Second RBM Layer on New Data ---
Layer 2 Epoch 50, Loss: 0.4647
Layer 2 Epoch 100, Loss: 0.4760
Layer 2 Epoch 150, Loss: 0.4620
Layer 2 Epoch 200, Loss: 0.4850
Second RBM Layer Training finished.

--- Building and Training DBN Classification Model for New Data ---

New Classification Model Summary:


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dbn_features_new                │ (None, 3)              │             0 │
│ (DBNFeatureExtractor_New)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier_new (Dense)          │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.4801 - loss: 0.8045 - val_accuracy: 0.5500 - val_loss: 0.7422
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4881 - loss: 0.7924 - val_accuracy: 0.5500 - val_loss: 0.7395
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4395 - loss: 0.8318 - val_accuracy: 0.5500 - val_loss: 0.7368
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4708 - loss: 0.7985 - val_accuracy: 0.5500 - val_loss: 0.7343
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4499 - loss: 0.8164 - val_accuracy: 0.5500 - val_loss: 0.7318
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3715 - loss: 0.8738 - val_accuracy: 0.5500 - val_loss: 0.7294
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4625 - loss: 0.7940 - val_accuracy: 0.5500 - val_loss: 0.7273
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4477 - loss: 0.8051 - val_accuracy: 0.5500 - val_loss:


Classification Accuracy on new test data: 0.6500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

Sample Data for Classification (New Dataset):
[[1. 1. 1. 1. 0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1. 1. 1. 1. 1.]]

Predictions (Probability of Class 1) on New Dataset:
[[0.5003126]
 [0.5050555]
 [0.5026694]
 [0.5035267]
 [0.5031961]]

Binary Predictions on New Dataset:
[[1]
 [1]
 [1]
 [1]
 [1]]


In [11]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Reuse the original data and parameters for consistency
# data (from previous RBM/DBN examples)
# num_visible (input size)
# num_hidden_layer1 (size of first hidden layer in previous DBN)
# num_hidden_layer2 (size of second hidden layer in previous DBN)
# classification_labels (from previous DBN classification example)

# Ensure data and labels are available (assuming previous cells were run)
if 'data' not in locals() or 'classification_labels' not in locals():
    print("Please run the previous cells to load data and labels.")
else:
    print("Using existing data and labels.")
    print("Data shape:", data.shape)
    print("Labels shape:", classification_labels.shape)

    # Define the Keras Sequential model
    # Input layer size = num_visible
    # First Dense layer size = num_hidden_layer1 (first RBM hidden size)
    # Second Dense layer size = num_hidden_layer2 (second RBM hidden size)
    # Output Dense layer size = 1 (for binary classification)

    model = keras.Sequential([
        # Input layer (implicitly defined by the first layer's input_shape)
        layers.Dense(num_hidden_layer1, activation='sigmoid', input_shape=(num_visible,), name='hidden_layer_1'),
        layers.Dense(num_hidden_layer2, activation='sigmoid', name='hidden_layer_2'),
        layers.Dense(1, activation='sigmoid', name='output_layer') # Binary classification output
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("\nKeras DBN-like Model Summary:")
    model.summary()

    # --- Training the Keras Model ---
    print("\n--- Training Keras DBN-like Model ---")

    epochs_keras = 500 # Adjust epochs
    batch_size_keras = 10

    # Split data for training (optional but good practice)
    # train_data_keras, test_data_keras, train_labels_keras, test_labels_keras = train_test_split(
    #     data, classification_labels, test_size=0.2, random_state=42)

    # For simplicity with this small dataset, we'll train on the full data for now
    history_keras = model.fit(data, classification_labels,
                              epochs=epochs_keras,
                              batch_size=batch_size_keras,
                              verbose=1)

    print("\nKeras DBN-like Model Training finished.")

    # --- Evaluation ---
    loss_keras, accuracy_keras = model.evaluate(data, classification_labels, verbose=0)
    print(f'\nClassification Accuracy on training data (Keras model): {accuracy_keras:.4f}')

    # --- Make Predictions ---
    sample_data_keras = np.array([
        [1, 1, 0, 0, 0, 0], # Should be class 1
        [0, 0, 1, 1, 1, 0], # Should be class 0
        [1, 0, 0, 0, 0, 0]  # Should be class 1
    ], dtype=np.float32)

    predictions_keras = model.predict(sample_data_keras)

    print("\nSample Data for Classification (Keras Model):")
    print(sample_data_keras)
    print("\nPredictions (Probability of Class 1) on Keras Model:")
    print(predictions_keras)

    # Convert probabilities to binary predictions (0 or 1)
    binary_predictions_keras = (predictions_keras > 0.5).astype(int)
    print("\nBinary Predictions on Keras Model:")
    print(binary_predictions_keras)

Using existing data and labels.
Data shape: (6, 6)
Labels shape: (6, 1)

Keras DBN-like Model Summary:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 2)              │            14 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 5)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35 (140.00 B)

 Trainable params: 35 (140.00 B)

 Non-trainable params: 0 (0.00 B)


--- Training Keras DBN-like Model ---
Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.7019
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.5000 - loss: 0.7016
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 0.7013
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5000 - loss: 0.7011
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 0.7008
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5000 - loss: 0.7006
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5000 - loss: 0.7004
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.5000 - loss: 0.7001
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 0.6999
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.6997
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.6995
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━


Classification Accuracy on training data (Keras model): 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

Sample Data for Classification (Keras Model):
[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]

Predictions (Probability of Class 1) on Keras Model:
[[0.5866257]
 [0.4212281]
 [0.5036831]]

Binary Predictions on Keras Model:
[[1]
 [0]
 [1]]
